In [1]:
library(tidyverse)
library(DESeq2)
library(BiocParallel)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects

In [2]:
n_cores <- detectCores()
BiocParallel::register(MulticoreParam(n_cores))

## Constants

In [3]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
dsets <- c("unified_cervical_data")
dset_paths <- unlist(map(dsets, function(d) paste0(dirs$data_dir, "/", d)))
matrisome_list <- matrisome_list <- paste(dirs$data_dir, "matrisome", "matrisome_hs_masterlist.tsv", sep = "/")
dset_idx <- 1
                         
padj_thresh <- 0.05

## Functions

In [4]:
run_DESeq_and_get_results <- function(dds) {
    #We want results without outlier removal or independent filtering since filtering should happen downstream.
    #See docs: https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#how-can-i-get-unfiltered-deseq2-results
    dds_seq <- DESeq(dds, parallel = TRUE)
    res <- results(
        dds_seq,
        contrast = c("condition", "tumor", "healthy"),
        pAdjustMethod = "BH",
        alpha = padj_thresh,
        parallel = TRUE
    )
    return(as_tibble(res, rownames = "geneID"))
}

## Read in data

In [5]:
counts <- read_tsv(paste0(dset_paths[dset_idx], "/counts.tsv")) %>%
    select(-"Entrez_Gene_Id") %>%
    mutate_if(is.numeric, round, 0) %>%
    column_to_rownames(var = "Hugo_Symbol")
coldata <- read_tsv(paste0(dset_paths[dset_idx], "/coldata.tsv")) %>%
    column_to_rownames(var = "sample_name")
all(rownames(coldata) == colnames(counts))

Parsed with column specification:
cols(
  .default = col_double(),
  Hugo_Symbol = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  sample_name = col_character(),
  condition = col_character(),
  data_source = col_character()
)


[1] TRUE

In [6]:
sum(coldata$condition == "healthy")
sum(coldata$condition == "tumor")

[1] 13

[1] 259

## RUN DGE Analysis

- `dds_naive`: measure the effect of `condition`
- `dds_informed`: measure the effect of `condition`, controlling for `data_source` (batch effect)

See docs: https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#quick-start

In [7]:
# dds_naive <- DESeqDataSetFromMatrix(
#     countData = counts,
#     colData = coldata,
#     design = ~ condition
# )

dds_informed <- DESeqDataSetFromMatrix(
    countData = counts,
    colData = coldata,
    design = ~ data_source + condition
)

converting counts to integer mode
Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”

In [8]:
# dge_res_naive_df <- run_DESeq_and_get_results(dds_naive)
dge_res_informed_df <- run_DESeq_and_get_results(dds_informed)

estimating size factors
estimating dispersions
gene-wise dispersion estimates: 16 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 16 workers
-- replacing outliers and refitting for 2020 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


### How many significant DEGs? (p-value < `padj_thresh`)

In [9]:
# sig_dge_res_naive_df <- dplyr::filter(dge_res_naive_df, padj < padj_thresh)
sig_dge_res_informed_df <- dplyr::filter(dge_res_informed_df, padj < padj_thresh)
# nrow(sig_dge_res_naive_df)
nrow(sig_dge_res_informed_df)

[1] 4694

## Save results

In [10]:
write_tsv(dge_res_informed_df, paste0(dirs$analysis_dir, "/", dsets[dset_idx], "_unfiltered_DESeq_results.tsv"))